In [3]:
!pip install -U -q sagemaker
!pip install -q tensorflow==2.8.0
!pip install "h5py==2.10.0"

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.23.23 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python

In [13]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
tf_framework_version = tf.__version__
import h5py
import numpy as np
import os
import tarfile
from sagemaker.tensorflow.serving import Model

In [5]:
# input_bucket = "sagemaker-us-east-1-780374501202/"
# prefix = "Model"
# filename = "modelA"
!aws s3 cp s3://"sagemaker-us-east-1-780374501202/Model/modelA.zip" "modelA.zip"
# Run the following conda install in case unzip is not found
!conda install -y -c conda-forge unzip
!unzip modelA.zip -d keras_model

download: s3://sagemaker-us-east-1-780374501202/Model/modelA.zip to ./modelA.zip
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - unzip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.10.8          |   py37h89c1867_2         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    openssl-1.1.1n             |       h7f8727e_0         2.5 MB
    unzip-6.0                  |       h516909a_2         141 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.8 MB

The following NEW packages will be INSTALLED:

  unzip              conda-forge/linux-64::unzip-6.0-h516909a_2

The following packages will be UPDATED:

  certifi                          2021.10.8-py37h89c1867_1 --> 2021.1

In [15]:
with open(os.path.join('keras_model/model', 'model.json'), 'r') as fp:
    loaded_model_json = fp.read()
loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights('keras_model/model/model.h5')
loaded_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 90, 128)           92160     
                                                                 
 lstm_21 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_5 (RepeatVect  (None, 90, 64)           0         
 or)                                                             
                                                                 
 lstm_22 (LSTM)              (None, 90, 64)            33024     
                                                                 
 lstm_23 (LSTM)              (None, 90, 128)           98816     
                                                                 
 time_distributed_5 (TimeDis  (None, 90, 51)           6579      
 tributed)                                            

In [9]:
model_version = '1'
export_dir = 'export/Servo/' + model_version
tf.saved_model.save(loaded_model, export_dir)
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True)
model_data = sess.upload_data(path=model_archive, key_prefix='model')

INFO:tensorflow:Assets written to: export/Servo/1/assets


In [16]:
instance_type = 'ml.t2.medium'
sm_model = Model(model_data=model_data, framework_version=tf_framework_version, role=role)
predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type)

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------!

In [11]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'tensorflow-inference-2022-04-06-01-46-09-383'

In [ ]:
predictor